In [1]:
#Reference: https://www.kaggle.com/anandsingh1011/flower-recognition-model-in-keras

import numpy as np 
import pandas as pd 

# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.optimizers import Adam
from glob import glob
import os

Using TensorFlow backend.


In [2]:
training_dir = "/Users/vineevineela/Documents/Semester-4/CMPE-256/Project/dataset/training"
test_dir = "/Users/vineevineela/Documents/Semester-4/CMPE-256/Project/dataset/validation"

In [3]:
# useful for getting number of files
train_count = len(glob(training_dir + '/*/*.jp*g'))
test_count = len(glob(test_dir + '/*/*.jp*g'))
print(train_count)
print(test_count)

7699
850


In [4]:
# getting the number of classes 
folders = glob(training_dir + '/*')
num_classes = len(folders)
print (num_classes)

5


In [5]:
classifier = Sequential()

In [6]:
input_size = (256, 256)
batch_size = 600

classifier.add(Conv2D(32, (3, 3), input_shape=(256,256,3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2), dim_ordering="th"))


classifier.add(Conv2D(32, (3, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))

classifier.add(Conv2D(64, (3, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))

classifier.add(Flatten())

classifier.add(Dense(units=64, activation='relu'))
classifier.add(Dropout(0.5))
classifier.add(Dense(units = num_classes, activation='softmax'))

/Users/vineevineela/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), data_format="channels_first")`
  """


In [7]:
opt = Adam(lr=1e-3, decay=1e-6)
classifier.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import preprocess_input

training_datagen = ImageDataGenerator(
                                    rescale=1./255,   # all pixel values will be between 0 an 1
                                    shear_range=0.2, 
                                    zoom_range=0.2,
                                    horizontal_flip=True,
                                    preprocessing_function=preprocess_input)

test_datagen = ImageDataGenerator(
                                    rescale=1./255,   # all pixel values will be between 0 an 1
                                    preprocessing_function=preprocess_input)


training_generator = training_datagen.flow_from_directory(training_dir, target_size = input_size, batch_size = batch_size, class_mode = 'categorical')
test_generator = test_datagen.flow_from_directory(test_dir, target_size = input_size, batch_size = batch_size, class_mode= 'categorical')


Found 7721 images belonging to 5 classes.
Found 856 images belonging to 5 classes.


In [9]:
test_generator.class_indices

{'drawings': 0,
 'engraving': 1,
 'iconography': 2,
 'painting': 3,
 'sculpture': 4}

In [10]:
classifier.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 254, 254, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 254, 127, 16)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 252, 125, 32)      4640      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 126, 62, 32)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 124, 60, 64)       18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 62, 30, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 119040)            0         
__________

In [ ]:
model_info = classifier.fit_generator(training_generator,
                         steps_per_epoch=train_count/batch_size,
                         epochs=2,
                         validation_data=test_generator,
                         validation_steps=test_count/batch_size,
                         workers=12)

Epoch 1/2


/Users/vineevineela/anaconda2/envs/py36/lib/python3.6/site-packages/PIL/Image.py:931: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')
